In [1]:
import pandas as pd
import yfinance as yf
import openpyxl
import json
import os
from dataclasses import dataclass
import matplotlib as plt
from datetime import datetime

In [2]:
@dataclass
class GraphPoint:
    index: int
    high: float
    low: float
    close: float
    volume: float
    date: str

In [3]:
@dataclass
class FilteredSymbols:
    latestPrice: float
    latestVolume: int
    fiftyDMA: float
    twoHundredDMA: float
    symbol: str
    recoveryPercentage: float

In [4]:
def shortListed500CrStocks(df, rowName: str, marketCapInCrs: float):
    return df[df[rowName] >= marketCapInCrs]

In [5]:
df = pd.read_excel('MCAP28032024.xlsx')
df = df.iloc[:-2, :]
df['Symbol'] = df['Symbol'] + '.NS'
df['Market capitalization as on March 28, 2024\n(In Cr.)']
filtered_df = shortListed500CrStocks(df,'Market capitalization as on March 28, 2024\n(In Cr.)', 500.0 )
filtered_df

,Sr. No.,Symbol,Company Name,"Market capitalization as on March 28, 2024\n(In lakhs)","Market capitalization as on March 28, 2024\n(In Cr.)"
0,1.0,RELIANCE.NS,Reliance Industries Limited,201056022.448876,2.010560e+06
1,2.0,TCS.NS,Tata Consultancy Services Limited,140247926.460234,1.402479e+06
2,3.0,HDFCBANK.NS,HDFC Bank Limited,109991524.325625,1.099915e+06
3,4.0,ICICIBANK.NS,ICICI Bank Limited,76765684.662095,7.676568e+05
4,5.0,BHARTIARTL.NS,Bharti Airtel Limited,69478399.828022,6.947840e+05
...,...,...,...,...,...
1412,1413.0,DONEAR.NS,Donear Industries Limited,50622,5.062200e+02
1413,1414.0,CAREERP.NS,Career Point Limited,50467.212786,5.046721e+02
1414,1415.0,INTLCONV.NS,International Conveyors Limited,50386.305,5.038630e+02
1415,1416.0,SHREEPUSHK.NS,Shree Pushkar Chemicals & Fertilisers Limited,50206.0845,5.020608e+02


In [6]:
filtered_tickers_symbol_list = filtered_df['Symbol'].astype(str).tolist()
filtered_tickers_symbol = ' '.join(filtered_tickers_symbol_list)

In [7]:
def getAllTimeHighPriceAndItsDateInLastVariableDays(df, prevNumberOfDays):
    # print(df)
    if(df.shape[0] < prevNumberOfDays):
        high_filtered_df = df['High']
    else:
        high_filtered_df = df.iloc[-prevNumberOfDays:, :]['High']
    # high_filtered_df.plot()
    high_filtered_df_high_index = high_filtered_df.idxmax() 
    return GraphPoint(high_filtered_df_high_index, high_filtered_df.max(), df.loc[high_filtered_df_high_index,'Low'], df.loc[high_filtered_df_high_index,'Close'], df.loc[high_filtered_df_high_index,'Volume'], df.loc[high_filtered_df_high_index,'Date'])

In [8]:
def getLowestPointAfterIndex(index: int, df):
    df_slice = df.iloc[index:, :]['Low']
    df_slice_low_point_index = df_slice.idxmin()
    return GraphPoint(df_slice_low_point_index,  df.loc[df_slice_low_point_index, 'High'], df_slice.min() , df.loc[df_slice_low_point_index, 'Close'], df.loc[df_slice_low_point_index, 'Volume'], df.loc[df_slice_low_point_index, 'Date'])

In [9]:
def getLatestClosePoint(symbol_max_history):
    return  GraphPoint(
        symbol_max_history.shape[0] - 1,
        symbol_max_history.iloc[-1]['High'],
        symbol_max_history.iloc[-1]['Low'],
        symbol_max_history.iloc[-1]['Close'],
        symbol_max_history.iloc[-1]['Volume'],
        symbol_max_history.iloc[-1]['Date'])

In [10]:
tickers = yf.Tickers(filtered_tickers_symbol)
sbin_ticker = tickers.tickers['RELIANCE.NS']

sbin_history = sbin_ticker.history(period="max")
sbin_history = sbin_history.reset_index(drop=False)
    
all_time_high_price = sbin_history['High'].max() 
highest_point_in_last_variable_days = getAllTimeHighPriceAndItsDateInLastVariableDays(sbin_history, 125)
print(highest_point_in_last_variable_days)

lowest_point_after_hitting_high = getLowestPointAfterIndex(highest_point_in_last_variable_days.index, sbin_history)
print(lowest_point_after_hitting_high)

GraphPoint(index=7159, high=3197.0, low=3096.0, close=3177.25, volume=6134855, date=Timestamp('2024-07-05 00:00:00+0530', tz='Asia/Kolkata'))
GraphPoint(index=7159, high=3197.0, low=3096.0, close=3177.25, volume=6134855, date=Timestamp('2024-07-05 00:00:00+0530', tz='Asia/Kolkata'))


In [11]:
def currValueGt(currVol: float, valueToCompareWith: float):
    return currVol > valueToCompareWith

def currValuePercentageChangeGt(currVol: float, valueToCompareWith: float, thresholdChange: float):
    return ((currVol/valueToCompareWith) > thresholdChange)

In [12]:
def getRecoveryPercentage(high_point: GraphPoint, lowest_piont_after_high: GraphPoint, latest_close_point: GraphPoint):
    return (latest_close_point.close - lowest_piont_after_high.low)/(high_point.high - lowest_piont_after_high.low)

In [13]:
numberOfDays = {"1d" : 1, "5d" : 5, "1mo" : 30 , "3mo" : 90, "6mo" : 180 , "1y": 360 , "2y": 720, "5y" : 1800 , "10y": 3600, "max": "max"}

In [14]:
def calculateAverageVolumeinLastXDays(numOfDays: str, tickerInfo):
    ticker_history_for_x_days = tickerInfo.history(period=numOfDays)
    return ticker_history_for_x_days['Volume'].sum() / numberOfDays[numOfDays]
    

In [15]:
latest_close_point = GraphPoint(sbin_history.shape[0] - 1, sbin_history.iloc[-1]['High'], sbin_history.iloc[-1]['Low'], sbin_history.iloc[-1]['Close'], sbin_history.iloc[-1]['Volume'], sbin_history.iloc[-1]['Date'])
getRecoveryPercentage(highest_point_in_last_variable_days, lowest_point_after_hitting_high, latest_close_point)
# currValueGt(0.9, getRecoveryPercentage(highest_point_in_last_variable_days, lowest_point_after_hitting_high, latest_close_point))

0.8044554455445545

In [16]:
def getLastXDaysVolumeAverage(numOfDays, ticker: yf.Ticker):
    if(numOfDays=="10d"):
        return ticker.info['averageVolume10days']
    else:
        return calculateAverageVolumeinLastXDays(numOfDays, ticker)

In [17]:
def getLastXDaysPriceAverage(numOfDays, tickerInfo: yf.Ticker):
    if(numOfDays=="10d"):
        return tickerInfo.info['averageVolume10days']
    else:
        return calculateAverageVolumeinLastXDays(numOfDays, tickerInfo)

In [18]:
# fifty_day_average_price = sbin_ticker.info['fiftyDayAverage']
# two_hundred_day_average_price = sbin_ticker.info['twoHundredDayAverage']

In [19]:
# getLastXDaysVolumeAverage("1d", sbin_ticker)

In [20]:
tickers = yf.Tickers(filtered_tickers_symbol)
print(filtered_tickers_symbol)
symbols_filtered = []
for symbol in filtered_tickers_symbol_list:
    print(f'\n [{datetime.now().time()}]')
    try:
        symbol_ticker = tickers.tickers[symbol]
        symbol_max_history = symbol_ticker.history(period="max")
        symbol_max_history = symbol_max_history.reset_index(drop=False)
        all_time_high = symbol_max_history['High'].max()
        fifty_day_average_price = symbol_ticker.info['fiftyDayAverage']
        two_hundred_day_average_price = symbol_ticker.info['twoHundredDayAverage']
        highest_point_in_last_variable_days = getAllTimeHighPriceAndItsDateInLastVariableDays(symbol_max_history, 125)
        lowest_point_after_hitting_high = getLowestPointAfterIndex(highest_point_in_last_variable_days.index, symbol_max_history)
        latest_close_point = getLatestClosePoint(symbol_max_history)
        recovery_percentage = getRecoveryPercentage(highest_point_in_last_variable_days, lowest_point_after_hitting_high, latest_close_point)
        last_5d_vol_average = getLastXDaysVolumeAverage("5d", symbol_ticker)
        last_1mo_vol_average = getLastXDaysVolumeAverage("1mo", symbol_ticker)
        current_volume = symbol_ticker.info['volume']
        print(f'{symbol} evaluated')
        if(
            currValueGt(recovery_percentage, 0.9) 
            and currValuePercentageChangeGt(current_volume, last_5d_vol_average, 2.0) 
            and currValueGt(current_volume, last_1mo_vol_average) 
            and currValueGt(latest_close_point.close, fifty_day_average_price) 
            and currValueGt(latest_close_point.close, two_hundred_day_average_price)):
            print(f'recovery_percentage : {recovery_percentage}\ncurr_volume : {current_volume}\nlast_5d_vol_average : {last_5d_vol_average}\nlast_1mo_vol_average : {last_1mo_vol_average}\nlatest_close_point : {latest_close_point.close}\nfifty_day_average_price : {fifty_day_average_price}\ntwo_hundred_day_average_price : {two_hundred_day_average_price}')
            print(f'----------------------------------------\n{symbol} found worthy to look into :)\n----------------------------------------')
            symbols_filtered.append(FilteredSymbols(latest_close_point.close, latest_close_point.volume, fifty_day_average_price, two_hundred_day_average_price, symbol, recovery_percentage))
    except:
        print(f'Data not found in yFinance for : {symbol}')
        continue
print(symbols_filtered)

RELIANCE.NS TCS.NS HDFCBANK.NS ICICIBANK.NS BHARTIARTL.NS SBIN.NS INFY.NS LICI.NS ITC.NS HINDUNILVR.NS LT.NS BAJFINANCE.NS HCLTECH.NS MARUTI.NS SUNPHARMA.NS ADANIENT.NS KOTAKBANK.NS TITAN.NS ONGC.NS TATAMOTORS.NS NTPC.NS AXISBANK.NS DMART.NS ADANIGREEN.NS ADANIPORTS.NS ULTRACEMCO.NS ASIANPAINT.NS COALINDIA.NS BAJAJFINSV.NS BAJAJ-AUTO.NS POWERGRID.NS NESTLEIND.NS WIPRO.NS M&M.NS IOC.NS JIOFIN.NS HAL.NS DLF.NS ADANIPOWER.NS JSWSTEEL.NS TATASTEEL.NS SIEMENS.NS IRFC.NS VBL.NS ZOMATO.NS PIDILITIND.NS GRASIM.NS SBILIFE.NS BEL.NS LTIM.NS TRENT.NS PNB.NS INDIGO.NS BANKBARODA.NS HDFCLIFE.NS ABB.NS BPCL.NS PFC.NS GODREJCP.NS TATAPOWER.NS HINDALCO.NS HINDZINC.NS TECHM.NS AMBUJACEM.NS INDUSINDBK.NS CIPLA.NS GAIL.NS RECLTD.NS BRITANNIA.NS UNIONBANK.NS ADANIENSOL.NS IOB.NS LODHA.NS EICHERMOT.NS CANBK.NS TATACONSUM.NS DRREDDY.NS TVSMOTOR.NS ZYDUSLIFE.NS ATGL.NS VEDL.NS CHOLAFIN.NS HAVELLS.NS HEROMOTOCO.NS DABUR.NS SHREECEM.NS MANKIND.NS BAJAJHLDNG.NS DIVISLAB.NS APOLLOHOSP.NS NHPC.NS SHRIRAMFIN.NS BO

RELIANCE.NS evaluated

 [23:19:01.838324]
TCS.NS evaluated

 [23:19:02.861272]
HDFCBANK.NS evaluated

 [23:19:03.943042]
ICICIBANK.NS evaluated

 [23:19:04.974865]
BHARTIARTL.NS evaluated

 [23:19:05.922677]
SBIN.NS evaluated

 [23:19:07.111574]
INFY.NS evaluated

 [23:19:08.203336]
LICI.NS evaluated

 [23:19:08.957769]
ITC.NS evaluated

 [23:19:10.159770]
HINDUNILVR.NS evaluated

 [23:19:12.671172]
LT.NS evaluated

 [23:19:13.685559]
BAJFINANCE.NS evaluated

 [23:19:14.686650]
HCLTECH.NS evaluated

 [23:19:15.629257]
MARUTI.NS evaluated

 [23:19:16.550643]
SUNPHARMA.NS evaluated

 [23:19:17.723839]
ADANIENT.NS evaluated

 [23:19:18.775983]
KOTAKBANK.NS evaluated

 [23:19:19.722608]
TITAN.NS evaluated

 [23:19:20.730528]
ONGC.NS evaluated
recovery_percentage : 0.9320943649596313
curr_volume : 39781962
last_5d_vol_average : 19714704.2
last_1mo_vol_average : 14417708.433333334
latest_close_point : 288.20001220703125
fifty_day_average_price : 272.925
two_hundred_day_average_price : 241.35

$L&TFH.NS: possibly delisted; No timezone found
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/L&TFH.NS?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=L%26TFH.NS&crumb=%2FiBp5L9X1WZ


Data not found in yFinance for : L&TFH.NS

 [23:21:53.855618]
UNOMINDA.NS evaluated

 [23:21:55.026744]
PAGEIND.NS evaluated

 [23:21:55.908129]
MRPL.NS evaluated

 [23:21:56.888373]
AUBANK.NS evaluated

 [23:21:57.749999]
MAZDOCK.NS evaluated

 [23:21:58.472489]
HUDCO.NS evaluated

 [23:21:59.268452]
GUJGASLTD.NS evaluated

 [23:22:00.087639]
NIACL.NS evaluated

 [23:22:00.911797]
CRISIL.NS evaluated

 [23:22:01.962448]
AIAENG.NS evaluated

 [23:22:02.891438]
FEDERALBNK.NS evaluated

 [23:22:04.167469]
IREDA.NS evaluated

 [23:22:04.853186]
VOLTAS.NS evaluated

 [23:22:05.890597]
DALBHARAT.NS evaluated

 [23:22:06.687461]
POONAWALLA.NS evaluated

 [23:22:07.670862]
MEDANTA.NS evaluated

 [23:22:08.347799]
IRB.NS evaluated

 [23:22:09.233565]
3MINDIA.NS evaluated

 [23:22:10.087842]
MFSL.NS evaluated

 [23:22:11.002351]
M&MFIN.NS evaluated

 [23:22:12.032279]
UPL.NS evaluated

 [23:22:13.020595]
HONAUT.NS evaluated

 [23:22:14.021373]
BSE.NS evaluated

 [23:22:14.804283]
FLUOROCHEM.NS 

$UJJIVAN.NS: possibly delisted; No price data found  (period=5d)
$UJJIVAN.NS: possibly delisted; No price data found  (period=1mo)


...

Data not found in yFinance for : ACCENTMIC.NS

 [23:40:14.920017]
PTL.NS evaluated

 [23:40:16.072733]
INDOBORAX.NS evaluated

 [23:40:16.822727]
MOLDTECH.NS evaluated

 [23:40:17.691140]
MAZDA.NS evaluated

 [23:40:18.563702]
MINDTECK.NS evaluated

 [23:40:19.381490]


$COOLCAPS.NS: possibly delisted; No timezone found


Data not found in yFinance for : COOLCAPS.NS

 [23:40:22.139103]


$ALLETEC.NS: possibly delisted; No timezone found


Data not found in yFinance for : ALLETEC.NS

 [23:40:24.162421]
20MICRONS.NS evaluated

 [23:40:25.164300]
VIKASECO.NS evaluated

 [23:40:26.137664]
ORIENTBELL.NS evaluated

 [23:40:27.148955]
DONEAR.NS evaluated

 [23:40:28.172161]
CAREERP.NS evaluated

 [23:40:29.116709]
INTLCONV.NS evaluated

 [23:40:29.888130]
SHREEPUSHK.NS evaluated

 [23:40:30.747990]
PRITIKAUTO.NS evaluated
[FilteredSymbols(latestPrice=288.20001220703125, latestVolume=39781962, fiftyDMA=272.925, twoHundredDMA=241.357, symbol='ONGC.NS', recoveryPercentage=0.9320943649596313), FilteredSymbols(latestPrice=922.0, latestVolume=466700, fiftyDMA=665.943, twoHundredDMA=441.95325, symbol='AEGISCHEM.NS', recoveryPercentage=0.9238579152014623), FilteredSymbols(latestPrice=848.7000122070312, latestVolume=569235, fiftyDMA=716.779, twoHundredDMA=628.698, symbol='SPLPETRO.NS', recoveryPercentage=0.9057354557641829), FilteredSymbols(latestPrice=187.99000549316406, latestVolume=17455116, fiftyDMA=161.4084, twoHundredDMA=150.6553

In [23]:
[FilteredSymbols(latestPrice=288.20001220703125, latestVolume=39781962, fiftyDMA=272.925, twoHundredDMA=241.357, symbol='ONGC.NS', recoveryPercentage=0.9320943649596313), FilteredSymbols(latestPrice=922.0, latestVolume=466700, fiftyDMA=665.943, twoHundredDMA=441.95325, symbol='AEGISCHEM.NS', recoveryPercentage=0.9238579152014623), FilteredSymbols(latestPrice=848.7000122070312, latestVolume=569235, fiftyDMA=716.779, twoHundredDMA=628.698, symbol='SPLPETRO.NS', recoveryPercentage=0.9057354557641829), FilteredSymbols(latestPrice=187.99000549316406, latestVolume=17455116, fiftyDMA=161.4084, twoHundredDMA=150.65535, symbol='MARKSANS.NS', recoveryPercentage=0.9500992274520421), FilteredSymbols(latestPrice=591.0, latestVolume=2975180, fiftyDMA=530.027, twoHundredDMA=540.664, symbol='UJJIVAN.NS', recoveryPercentage=0.9045215033645972), FilteredSymbols(latestPrice=1609.800048828125, latestVolume=210441, fiftyDMA=1253.108, twoHundredDMA=1072.2946, symbol='AURIONPRO.NS', recoveryPercentage=0.9969868875628141), FilteredSymbols(latestPrice=1790.199951171875, latestVolume=133347, fiftyDMA=1444.758, twoHundredDMA=1135.778, symbol='DHANUKA.NS', recoveryPercentage=0.9221291161758473), FilteredSymbols(latestPrice=127.43000030517578, latestVolume=3065454, fiftyDMA=101.7516, twoHundredDMA=95.51365, symbol='SUVEN.NS', recoveryPercentage=0.9659066321220517), FilteredSymbols(latestPrice=300.32000732421875, latestVolume=1836114, fiftyDMA=222.8834, twoHundredDMA=199.9561, symbol='WALCHANNAG.NS', recoveryPercentage=1.0)]


[FilteredSymbols(latestPrice=288.20001220703125, latestVolume=39781962, fiftyDMA=272.925, twoHundredDMA=241.357, symbol='ONGC.NS', recoveryPercentage=0.9320943649596313),
 FilteredSymbols(latestPrice=922.0, latestVolume=466700, fiftyDMA=665.943, twoHundredDMA=441.95325, symbol='AEGISCHEM.NS', recoveryPercentage=0.9238579152014623),
 FilteredSymbols(latestPrice=848.7000122070312, latestVolume=569235, fiftyDMA=716.779, twoHundredDMA=628.698, symbol='SPLPETRO.NS', recoveryPercentage=0.9057354557641829),
 FilteredSymbols(latestPrice=187.99000549316406, latestVolume=17455116, fiftyDMA=161.4084, twoHundredDMA=150.65535, symbol='MARKSANS.NS', recoveryPercentage=0.9500992274520421),
 FilteredSymbols(latestPrice=591.0, latestVolume=2975180, fiftyDMA=530.027, twoHundredDMA=540.664, symbol='UJJIVAN.NS', recoveryPercentage=0.9045215033645972),
 FilteredSymbols(latestPrice=1609.800048828125, latestVolume=210441, fiftyDMA=1253.108, twoHundredDMA=1072.2946, symbol='AURIONPRO.NS', recoveryPercentage=0

In [22]:
# tickers.tickers['AEGISCHEM.NS'].info['']
getLastXDaysVolumeAverage('10d', tickers.tickers['AEGISCHEM.NS'])

1101857